## Import

In [1]:
import os
import sys
os.chdir('../')
new_current_dir = os.getcwd()  # get new current working directory
print("New current directory:", new_current_dir)
sys.path.append('./src/python/')

# # import
from preamble.preamble import *
# from gamma.gamma import GammaNPP
# from gamma.rules import make_rules, Rules, readable_rules
# from gamma.common import npp_from_json, set_of_frozenset

New current directory: /home/fiftyfour/Documents/NetworkPricingGraphContraction


In [2]:
# import original problem results 
result_path = '/home/fiftyfour/Documents/Archive/experience_6/20241114_process_result/'
with open(os.path.join(result_path, 'result_original.pkl'), 'rb') as f:
    original_result = pickle.load(f)

## Process data

In [3]:
# create dataframe from original problem results
cols_header = list(original_result['g30-01'].keys())
print(cols_header)
cols_header.remove('edge')
rows_header = list(original_result.keys())

original_meta_data = [
                [r[k] for k in cols_header] for r in original_result.values()
            ]
original_meta_data = pd.DataFrame(original_meta_data, index=rows_header,columns=cols_header)
original_meta_data['finish'] = original_meta_data['finish'].astype(bool)
original_meta_data['finish'] = original_meta_data['finish'].astype(int)

['edge', 'obj_value', 'preprocess_time', 'solve_time', 'n_vertex', 'n_edge', 'n_tolled', 'finish', 'n', 'min_sl', 'max_sl', 'm', 'H1', 'H2', 'H3', 'H4', 'max_attemp', 'option', 'heuristic', 'cf1', 'cf2', 'cf3', 'cf4', 'cf5', 'cf6', 'cf7', 'cf8', 'cf9']


In [4]:
# the list of original problem name
pb_list = [
            'g45-03', 'g45-04', 'g30-08', 'g35-03', 'g30-10', 'g50-08', 'g45-10', 'g30-03', 'g45-02',
            'g35-07', 'g35-05', 'g50-02', 'g40-05', 'g35-06', 'g50-06', 'g30-02', 'g45-09', 'g35-09',
            'g50-10', 'g45-07', 'g40-10', 'g50-07', 'g40-01', 'g35-02', 'g35-04', 'g45-01', 'g45-05', 'g30-04',
            'g45-08', 'g50-03', 'g30-05', 'g40-08', 'g30-01', 'g40-03', 'g40-04', 'g40-02', 'g35-10',
            'g30-06', 'g50-01', 'g35-08', 'g35-01', 'g30-09', 'g40-09', 'g40-07', 'g50-05', 'g50-04', 'g40-06',
            'g50-09', 'g30-07', 'g45-06',
]
    
    
result_dict = {}
for problem_name in pb_list:
    o_idx = f'000000-000000-000000-000000-0-0-0-0-0-0-0-0-0-{problem_name}'
    result = {}
    with open(os.path.join(result_path, f'result_{problem_name}.pkl'), 'rb') as f:
        result[o_idx] = original_result[problem_name]
        result.update(pickle.load(f))
        
    rows_header = list(map(lambda x: x[0], result[o_idx]['edge']))
    cols_header = list(result.keys())
    edge_index_data = [      
                        [ r['edge'][i][1] for r in result.values() ] 
                        for i,_ in enumerate(rows_header)   
                    ]
    edge_value_data = [      
                        [ r['edge'][i][2] for r in result.values() ] 
                        for i,_ in enumerate(rows_header)   
                    ]
    
    flow_value_data = [      
                        [ r['edge'][i][3] for r in result.values() ] 
                        for i,_ in enumerate(rows_header)   
                    ]
    # Table of the form
    #  edge v1 v2 v3 v4 ....
    #   e1  o11
    #   e2  o21
    #   e3  ...
    # ....
    edge_index_df = pd.DataFrame(edge_index_data, index = rows_header, columns=cols_header)
    edge_value_df = pd.DataFrame(edge_value_data, index = rows_header, columns=cols_header)
    flow_value_df = pd.DataFrame(flow_value_data, index = rows_header, columns=cols_header)
    
    
    ## meta_data
    # df of the form
    #  pb v1 v2 v3 v4 ....
    #   p1  o11
    #   p2  o21
    #   p3  ...
    # ....
    cols_header = list(result[o_idx].keys())
    cols_header.remove('edge')
    rows_header = list(result.keys())
    
    meta_data = [
                    [r[k] for k in cols_header] for r in result.values()
                ]
    meta_data_df = pd.DataFrame(meta_data, index=rows_header,columns=cols_header)
    meta_data_df[['min_sl', 'max_sl', 'm', 'H4']] = meta_data_df[['min_sl', 'max_sl', 'm', 'H4']].astype(int)
    meta_data_df['finish'] = meta_data_df['finish'].astype(bool)
    meta_data_df['finish'] = meta_data_df['finish'].astype(int)

    # original problem objective, adjusted time, #tolled arcs, #vertex
    o_obj      = meta_data_df['obj_value'][o_idx]
    o_time     = meta_data_df['solve_time'][o_idx] + meta_data_df['preprocess_time'][o_idx]
    o_n_tolled = meta_data_df['n_tolled'][o_idx]
    o_n_vertex = meta_data_df['n_vertex'][o_idx]

    meta_data_df['optimal_gap']    = (o_obj - meta_data_df['obj_value'])/abs(o_obj)
    meta_data_df['absolute_error'] = abs(meta_data_df['obj_value'] - o_obj)
    meta_data_df['relative_error'] = (meta_data_df['obj_value'] - o_obj)/o_obj
    
    meta_data_df['adjusted_time']   = meta_data_df['solve_time'] + meta_data_df['preprocess_time']
    meta_data_df['time_increase']   = (meta_data_df['adjusted_time'] - o_time)/o_time
    meta_data_df['adjusted_time_2'] = meta_data_df['adjusted_time'] # copy for later

    remove_rows = []
    # add up the solve time in the contracted space to adjusted_time_2
    for index, row in meta_data_df.iterrows():

        if (index != o_idx) and (not '-x' in index) and (not 'bsl' in index):
            oid = index[:-1] + 'x'
            meta_data_df.loc[index, 'adjusted_time_2'] = row['adjusted_time'] + meta_data_df.loc[oid, 'adjusted_time']
        
        else:
            if (not '-x' in index) and (not 'bsl' in index):
                remove_rows.append(index)
    
    meta_data_df.drop(remove_rows, inplace=True)
    
    meta_data_df['adjusted_time_increase'] = (meta_data_df['adjusted_time_2']-o_time)/o_time
    meta_data_df['tolled_ratio']           = meta_data_df['n_tolled']/o_n_tolled
    meta_data_df['vertex_ratio']           = meta_data_df['n_vertex']/o_n_vertex

    # edge value
    col1 = edge_value_df.iloc[0:,0]
    col2 = edge_value_df.iloc[:, 1:].mean(axis=1)
    
    result = col2 - col1
    meta_data_df['edge_diff_mean'] = result.mean() 
    meta_data_df['edge_diff_std']  = result.std()

    # flow value
    col1 = flow_value_df.iloc[0:,0]
    col2 = flow_value_df.iloc[:, 1:].mean(axis=1)
    
    result = (col2 - col1)    
    meta_data_df['flow_diff_mean'] = result.mean() 
    meta_data_df['flow_diff_std']  = result.std()
    
    # option was later renamed : method
    # heuristic was later renamed : strategy
    columns = ['min_sl', 'max_sl', 'm', 'H4', 'option', 'heuristic']
    
    # class for label and plot 
    meta_data_df['Class'] = meta_data_df[columns[:-2:]].astype(str).apply('-'.join, axis=1) # min_sl-max_sl-m-H4
    meta_data_df['Class1'] = meta_data_df[columns[-3:]].astype(str).apply('-'.join, axis=1) # H4-option-heuristic
    meta_data_df['Class2'] = meta_data_df[columns].astype(str).apply('-'.join, axis=1)      # min_sl-max_sl-m-H4-option-heuristic
    meta_data_df['Class3'] = meta_data_df['Class2'].astype(str) + f'-{problem_name}'        # min_sl-max_sl-m-H4-option-heuristic-pb_name
    meta_data_df['Class4'] = meta_data_df[columns[-2::]].astype(str).apply('-'.join, axis=1) + f'-{problem_name}' # option-heuristic-pb_name
    meta_data_df['pb_name'] = problem_name

    # only meta_data_df is used, the rest is not saved
    result_dict[problem_name] = {
                                'edge_index_df':edge_index_df.copy(),
                                'edge_value_df':edge_value_df.copy(),
                                'flow_value_df':flow_value_df.copy(),
                                'meta_data_df':meta_data_df.copy()
    }

merged_df = pd.DataFrame()
for pb_name in result_dict:
    merged_df = pd.concat([merged_df, result_dict[pb_name]['meta_data_df']])
    
merged_df = merged_df.T.filter(regex=r'^(?!000000-000000-000000-000000-0-0-0-0-0-0-0-0-0-.*).*$').T
result_dict['combined'] = {
                                'edge_index_df':None,
                                'edge_value_df':None,
                                'flow_value_df':None,
                                'meta_data_df':merged_df.copy()
    }
# Export data
with open('compile_data.pkl', 'wb') as f:
    pickle.dump(result_dict, f)

In [2]:
with open('./compile_data.pkl', 'rb') as f:
    RESULTS = pickle.load(f)

## Figures

In [3]:
### Plot settings
import matplotlib.animation as animation
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as plt

df = RESULTS['combined']['meta_data_df']


### Color classes
color_map = 'Accent'


### Class
class_labels = df['Class'].unique()
cmap = plt.colormaps[color_map]  # Use the 'viridis' colormap
n_colors = len(class_labels)
colors = [cmap(i/n_colors) for i in range(n_colors)]
class_color_map = dict(zip(class_labels, colors))

### Option
option_labels = df['option'].unique()
cmap = plt.colormaps[color_map]  # Use the 'viridis' colormap
n_colors = len(option_labels)
colors = [cmap(i/n_colors) for i in range(n_colors)]
option_color_map = dict(zip(option_labels, colors))

### Heuristic
heuristic_labels = df['heuristic'].unique()
cmap = plt.colormaps[color_map]  # Use the 'viridis' colormap
n_colors = len(heuristic_labels)
colors = [cmap(i/n_colors) for i in range(n_colors)]
heuristic_color_map = dict(zip(heuristic_labels, colors))

### Problem
problem_labels = df['pb_name'].unique()
cmap = plt.colormaps[color_map]  # Use the 'viridis' colormap
n_colors = len(problem_labels)
colors = [cmap(i/n_colors) for i in range(n_colors)]
problem_color_map = dict(zip(problem_labels, colors))

fr_dict = {
    'optimal_gap' : "Écart d'optimalité relatif",
    'heuristic': "Heuristique de rétrorésolution",
    'option': "Heuristique de transformation",
    'Class' : 'Classe',
    'tolled_ratio' : "Ratio d'arc controllé",
    'adjusted_time_increase' : "Augmentation relative du temps de résolution total",
    'time_increase' : "Augmentation relative du temps de résolution"
}

params = {
    'backend': 'pdf',
    #'axes.labelsize': 12,
    # 'text.fontsize': 12,
    # 'legend.fontsize': 12,
    # 'xtick.labelsize': 10,
    # 'ytick.labelsize': 10,
    'text.usetex': True,
    'axes.unicode_minus': True,
    'font.family': 'serif',  # Use serif fonts
    'font.serif': ['Computer Modern Roman'],  # Specify LaTeX serif font
    'font.sans-serif': ['Computer Modern Sans Serif'],  # Optional
    'font.monospace': ['Computer Modern Typewriter']  # Optional
}

plt.rcParams.update(params)

In [5]:
### Quadrant plot (obsolete):
### quadrant 1 : Parameter Distribution
### quadrant 2 : Option Distribution
### quadrant 3 : Heuristic Distribution
### quadrant 4 : Problem Distribution ()


### The purpose is to look for a tendency when the optimal ratio is high
### Don't be fool by randomness
### Every transformation for each problem was randomly generated

if False:
    title_font_size = 32
    label_font_size = 24
    
    for h4 in [0, 1]:
        for opt_r in [0.05, 0.10, 0.25, 0.50]:
    
            conditions = [
                (df['m']!=0),                   # The number of equivalence classes 
                (df['max_sl']<=5),              # Maximum length one equivalence class can have
                (df['max_sl']>=2),              # Maximum length one equivalence class can have (0 mean arbitrary, so whe need a lower limit)
                (df['H4']==h4),                 # Contiguous hypothesis (1 or 0)
                (df['min_sl']>=2),              # Minimum length one none trivial equivalence class can have
                (df['heuristic']!='x'),         # Heuristic (1-2-3-4-5-x) (x is in the transformed space)
                (df['max_sl']==df['min_sl']),
            ]
            
            
            data_df = df[np.logical_and.reduce(conditions)]
            total_element = len(data_df)
            
            conditions.append(df['optimality_gap']<=opt_r)   # Optimal gap
            
            data_df = df[np.logical_and.reduce(conditions)]
            number_of_element = len(data_df)
            # print(f"Optimal ratio :{opt_r}, H4:{h4}")
            # print(f"Number of element\t: {number_of_element}")
            # print(f"Total number of element\t: {total_element}")
            
            # Create the subplots
            fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(30, 10))
            fig.subplots_adjust(left=0, right=0.95, bottom=0, top=1, wspace=0.015, hspace=0)
            
            
            # Plot 1 - Parameter Distribution when optimal ratio is over x%
            fracs = data_df['Class'].value_counts(normalize=True)
            mask = fracs > 0.01
            labels = fracs.index[mask].tolist()
            sizes = fracs[mask].tolist()
            
            # Add a category for classes less than 0.01
            mask_less_than_001 = fracs <= 0.01
            if mask_less_than_001.any():
                num_others = mask_less_than_001.sum()
                class_color_map[f'Others ({num_others})'] = 'gray'
                labels.append(f'Others ({num_others})')
                sizes.append(fracs[mask_less_than_001].sum())
            
            colors = [class_color_map[label] for label in labels]
            axs[0].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', textprops={'fontsize': label_font_size}, rotatelabels=True, pctdistance=0.85, labeldistance=1.05)
            if h4==1:
                axs[0].set_title(r'Paramètres (\texttt{min_sl}-\texttt{max_sl}-\texttt{m}-$\texttt{H}_4$)', fontsize=title_font_size, y=1.05)
            ####################################################################
            ####################################################################
            
            
            # Plot 2 - Option Distribution when optimal ratio is over x%
            fracs = data_df['option'].value_counts(normalize=True)
            mask = fracs > 0.0005
            labels = fracs.index[mask].tolist()
            sizes = fracs[mask].tolist()
            
            # Add a category for options less than 0.0005
            mask_less_than_0005 = fracs <= 0.0005
            if mask_less_than_0005.any():
                num_others = mask_less_than_0005.sum()
                option_color_map[f'Others ({num_others})'] = 'gray'
                labels.append(f'Others ({num_others})')
                sizes.append(fracs[mask_less_than_0005].sum())
            
            colors = [option_color_map[label] for label in labels]
            labels = [rf"\textbf{{{x}}}" for x in labels]
            axs[1].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', textprops={'fontsize': label_font_size}, rotatelabels=False, pctdistance=0.85, labeldistance=1.05)
    
            if h4==1:
                axs[1].set_title('Heuristique de transformation', fontsize=title_font_size, y=1.05)
            ####################################################################
            ####################################################################
            
            
            # Plot 3 - Heuristic Distribution when optimal ratio is over x%
            fracs = data_df['heuristic'].value_counts(normalize=True)
            mask = fracs > 0.01
            labels = fracs.index[mask].tolist()
            sizes = fracs[mask].tolist()
            
            # Add a category for heuristics less than 0.01
            mask_less_than_001 = fracs <= 0.01
            if mask_less_than_001.any():
                num_others = mask_less_than_001.sum()
                heuristic_color_map[f'Others ({num_others})'] = 'gray'
                labels.append(f'Others ({num_others})')
                sizes.append(fracs[mask_less_than_001].sum())
            
            colors = [heuristic_color_map[label] for label in labels]
            axs[2].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', textprops={'fontsize': label_font_size}, rotatelabels=False, pctdistance=0.85, labeldistance=1.05)
            if h4==1:
                axs[2].set_title('Heuristique de rétrorésolution', fontsize=title_font_size, y=1.05)
            ####################################################################
            ####################################################################
    
            if False:
                # Plot 4 - Problem Distribution when optimal ratio is over x%
                fracs = data_df['pb_name'].value_counts(normalize=True)
                mask = fracs > 0.01
                labels = fracs.index[mask].tolist()
                sizes = fracs[mask].tolist()
                
                # Add a category for problems less than 0.01
                mask_less_than_001 = fracs <= 0.01
                if mask_less_than_001.any():
                    num_others = mask_less_than_001.sum()
                    problem_color_map[f'Others ({num_others})'] = 'gray'
                    labels.append(f'Others ({num_others})')
                    sizes.append(fracs[mask_less_than_001].sum())
            
                colors = [problem_color_map[label] for label in labels]
                axs[3].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', textprops={'fontsize': label_font_size}, rotatelabels=True, pctdistance=0.85, labeldistance=1.05)
                axs[3].set_title('Problem Distribution', fontsize=title_font_size, y=1.05)
            ####################################################################
            ####################################################################
            
            # plt.tight_layout()
            plt.savefig(f'./result/quadrant_{int(round(opt_r*100,0))}_{h4}_{number_of_element}_{total_element}.pdf', bbox_inches='tight')
            plt.close()

In [33]:
### Pairplot (obsolete)
if False:
    title_font_size = 32
    label_font_size = 24
    
    for opt_r in [0.99]: #0.05, 0.10, 0.25, 0.50]:
        for option in ["rnd", "min", "avg", "max", "bsl"]:
            for h4 in [1, 0]:
                conditions = [
                    (df['m']!=0),                   # The number of equivalence classes 
                    (df['max_sl']<=5),              # Maximum length one equivalence class can have
                    (df['max_sl']>=2),              # Maximum length one equivalence class can have (0 mean arbitrary, so whe need a lower limit)
                    (df['H4']==h4),                 # Contiguous hypothesis (1 or 0)
                    (df['min_sl']>=2),              # Minimum length one none trivial equivalence class can have
                    (df['heuristic']!='x'),         # Heuristic (1-2-3-4-5-x) (x is in the transformed space)
                    (df['max_sl']==df['min_sl']),
                    (df['option']==option),         # Option when going back to the original problem
                ]
                
        
                conditions.append(df['optimality_gap']<=opt_r)   # Optimal ratio
                data_df = df[np.logical_and.reduce(conditions)]
                total_element = len(data_df)
                
                
                data_df = df[np.logical_and.reduce(conditions)]
                number_of_element = len(data_df)
                
                if number_of_element>=25:
           
                    # # Create a PairGrid
                    # pair_grid = sns.PairGrid(data_df, hue='Class')
                    
                    # # Map upper triangle with regplot
                    # pair_grid.map_upper(sns.regplot, scatter_kws={'s': 5})
                    
                    # # Map lower triangle with kdeplot
                    # pair_grid.map_lower(sns.kdeplot)
                    
                    # # Map diagonal with histplot
                    # pair_grid.map_diag(sns.histplot, kde=True)
                    
                    # # Add legend
                    # pair_grid.add_legend()
    
                    # Get unique classes and sort them
                    sorted_classes = sorted(data_df['Class'].unique())
                    pairplot = sns.pairplot(data_df,
                                vars=['optimality_gap', 'tolled_ratio', 'adjusted_time_increase'],
                                hue='Class',
                                hue_order = sorted_classes,
                                diag_kind='kde',
                                plot_kws=dict(s=5), # Set the size of the points to 5,
                                 # corner=True
                                 )
    
                    
                    # Set custom labels for each axis
                    for ax in pairplot.axes.flatten():
                        # Set x and y labels using the desired format
                        # ax.set_xlabel(ax.get_xlabel().replace("_", " ").title(), fontsize=10)
                        # ax.set_ylabel(ax.get_ylabel().replace("_", " ").title(), fontsize=10)
    
                        if ax.get_xlabel():
                            ax.set_xlabel(fr_dict[ax.get_xlabel()], fontsize=10)
                        if ax.get_ylabel():
                            ax.set_ylabel(fr_dict[ax.get_ylabel()], fontsize=10)
                    
                    # plt.tight_layout()
                    plt.show()
                    plt.savefig(f'./result/pairplot_{int(round(opt_r*100,0))}-{option}-{h4}-{number_of_element}_{total_element}.pdf', bbox_inches='tight')
                    plt.close()

/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_11164/3159984605.py:66: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt

In [7]:
### Boxplot

from matplotlib.lines import Line2D

##########################################  ##########################################
# RND - Shortest path          (RND - 1) #  # ZIP - Shortest path          (ZIP - 1) #
# RND - Lower bound            (RND - 2) #  # ZIP - Lower bound            (ZIP - 2) #
# RND - Upper bound            (RND - 3) #  # ZIP - Upper bound            (ZIP - 3) #
# RND - Adaptative lower bound (RND - 4) #  # ZIP - Adaptative lower bound (ZIP - 4) #
# RND - Adaptative upper bound (RND - 5) #  # ZIP - Adaptative upper bound (ZIP - 5) #
# RND - Comparative projection (RND - 6) #  # ZIP - Comparative projection (ZIP - 6) #
##########################################  ##########################################
 
##########################################  ##########################################
# MIN - Shortest path          (MIN - 1) #  # AVG - Shortest path          (AVG - 1) #
# MIN - Lower bound            (MIN - 2) #  # AVG - Lower bound            (AVG - 2) #
# MIN - Upper bound            (MIN - 3) #  # AVG - Upper bound            (AVG - 3) #
# MIN - Adaptative lower bound (MIN - 4) #  # AVG - Adaptative lower bound (AVG - 4) #
# MIN - Adaptative upper bound (MIN - 5) #  # AVG - Adaptative upper bound (AVG - 5) #
# MIN - Comparative projection (MIN - 6) #  # AVG - Comparative projection (AVG - 6) #
##########################################  ##########################################

##########################################  ##########################################
# MAX - Shortest path          (MAX - 1) #  # BSL - Shortest path          (MAX - 1) #
# MAX - Lower bound            (MAX - 2) #  # BSL - Lower bound            (MAX - 2) #
# MAX - Upper bound            (MAX - 3) #  # BSL - Upper bound            (MAX - 3) #
# MAX - Adaptative lower bound (MAX - 4) #  # BSL - Adaptative lower bound (MAX - 4) #
# MAX - Adaptative upper bound (MAX - 5) #  # BSL - Adaptative upper bound (MAX - 5) #
# MAX - Comparative projection (MAX - 6) #  #                                        #
##########################################  ##########################################

title_font_size   = 32
legend_label_size = 22
label_font_size   = 35
tick_font_size    = 22

hue_order = ['rnd','bsl','zip','min','avg','max']
order = ['1','2','3','4','5','6','x']

# xticklabels = [
#                'Shortest path (1)',
#                'Lower bound (2)', 
#                'Upper bound (3)', 
#                'Adapt. lower bound (4)', 
#                'Adapt. upper bound (5)', 
#                'Hybrid (6)', 
#                'Contracted space (x)'
# ]

xticklabels = [
               'Plus court chemin (1)',
               'Borne inférieure (2)', 
               'Borne supérieure (3)', 
               'Borne inférieure adapt. (4)', 
               'Borne supérieure adapt. (5)', 
               'Hybride (6)', 
               'Espace contracté (x)'
]

for yy in ['adjusted_time_increase', 'optimal_gap', 'time_increase']:
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 20))
    for i in [1, 0]:
        j = 0 if i==1 else 1
        sns.boxplot(data=df[df['H4']==i], x="heuristic", y=yy, hue="option", hue_order=hue_order, order=order, ax=ax[j], legend=i).set(xlabel='')

        ax[j].tick_params(which='major', width=1.0, length=30, axis='x', labelsize=tick_font_size)
        if i==0:
            ax[j].xaxis.set_ticks_position('top')  # Move x-ticks to the top
            ax[j].set_xticklabels(xticklabels,  rotation=45,ha='center', va='center')
            ax[j].tick_params(axis='x', pad=85)
        else:
            ax[j].set_xticklabels([])  # This removes the x-tick labels
            
        ax[j].tick_params(axis='y', labelsize=tick_font_size)  # Adjust the labelsize as needed
        if 'time' in yy:
            ax[j].axhline(y=0, color='red', linestyle='--')
            ax[j].axhline(y=-0.25, color='red', linestyle='--')
            ax[j].axhline(y=-0.5, color='red', linestyle='--')

        else:
            ax[j].axhline(y=0, color='red', linestyle='--')
            ax[j].axhline(y=0.25, color='red', linestyle='--')
            ax[j].axhline(y=0.5, color='red', linestyle='--')
            
        # ax[j].set_ylabel(fr_dict[yy], fontsize=label_font_size)
        ax[j].set_ylabel('')
        

        # Adding a text box with rounded corners
        textstr = f'$H_4={i}$'
        props = dict(boxstyle='round', facecolor='lightblue', alpha=0.5)
        
        # Place the text box at a specified position
        ax[j].text(0.015, 0.975, textstr, fontsize=legend_label_size, bbox=props, 
        ha='left', va='top', transform=ax[j].transAxes)  # Use axes coordinates

        # Add the legend, including the phantom label
        handles, labels = ax[j].get_legend_handles_labels()
        bf_labels = [rf"\textbf{{{x}}}" for x in labels]
        if i==1:
            ax[j].legend(handles=handles, labels=bf_labels, bbox_to_anchor=(0.5, 1.015), loc='lower center', ncol=len(hue_order) + 1, fontsize=legend_label_size)
        
        
        # To set the x-axis range
        if yy =='optimality_gap':
            ax[j].set_ylim(-0.6, 1.3)
            plt.ylim(ymin=-0.6, ymax=1.3)
                
        elif yy == 'adjusted_time_increase':
            plt.ylim(ymin=-1.05, ymax=1.5)
            ax[j].set_ylim(-1.05, 1.5)
            
        elif yy == 'time_increase':
            plt.ylim(ymin=-1.05, ymax=0.5)
            ax[j].set_ylim(-1.05, 0.5)
            
    fig.text(0.025, 0.5+0.05, fr_dict[yy], va='center', ha='center', rotation='vertical', fontsize=label_font_size)
    fig.text(0.5+0.05, 0.025, "Stratégies", va='center', ha='center', fontsize=label_font_size)
    plt.tight_layout()
    plt.subplots_adjust(left=0.1, bottom=0.05)
    plt.savefig(f'./result/boxplot_full_{yy}_H4.pdf')
    plt.close()

/tmp/ipykernel_7051/3582622266.py:71: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax[j].set_xticklabels(xticklabels,  rotation=45,ha='center', va='center')
/tmp/ipykernel_7051/3582622266.py:71: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax[j].set_xticklabels(xticklabels,  rotation=45,ha='center', va='center')
/tmp/ipykernel_7051/3582622266.py:71: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax[j].set_xticklabels(xticklabels,  rotation=45,ha='center', va='center')


## Tables

In [8]:
## Retrieving every transformation parametrisation use
import glob
def convert(x):
    idn1, idn2, idn3, n, min_sl, max_sl, m, H1, H2, H3, H4, max_attemp, pb_name1, pb_name2, *_ = x.split('-')
    return '-'.join((idn1, idn2, idn3, n, min_sl, max_sl, m, H4, max_attemp, pb_name1+'-'+pb_name2))



# List all directories in the specified folder
data_directory = "/home/fiftyfour/Documents/Archive/experience_6/full_data/home/hebjul/NetworkPricingGraphContraction/data/generated/problems/paper/"
subdirectories = [d for d in os.listdir(data_directory)]
subdirectories.remove('original')

# dictionary of probleme name and its transformation file 
problems_transformations = {pb_name: glob.glob(os.path.join(data_directory, pb_name, '*-P.pkl')) for pb_name in subdirectories}

transformation_ids = []
for (pb_name, files) in problems_transformations.items():
    for file in files:
        with open(file, 'rb') as f:
            idn1, idn2 = os.path.splitext(os.path.basename(file))[0].split('-')[:2:]
            # the unique id string to identify transformation where (idn1, idn2) is added for uniqueness
            # (idn1, idn2), idn3, n, min_sl, max_sl, m, H1, H2, H3, H4, max_attemp, pb_name  
            transformation_ids += list(map(lambda x: f'{idn1}-{idn2}-{x}', pickle.load(f).keys()))

trans_ids_split = [x.split('-') for x in transformation_ids]
all_trans_list = list(map(convert, transformation_ids))

####
success = RESULTS
best_case_scenario_trans_list = success[(success['heuristic']=='1') & (success['option']=='bsl')].index.tolist()
best_case_scenario_trans_list = list(map(convert, best_case_scenario_trans_list))


## Verify uniqueness
print(len(all_trans_list),len(set(all_trans_list)))
print(len(best_case_scenario_trans_list),len(set(best_case_scenario_trans_list)))

if len(all_trans_list) == len(set(all_trans_list)):
    print('all transformation are unique')
if len(best_case_scenario_trans_list) == len(set(best_case_scenario_trans_list)):
    print('all solve solve transformation are unique')
if len(set(all_trans_list)) == len(set(best_case_scenario_trans_list)):
    print('all generated transformation have been solved')

(2458685, 46)
71484 71484
71484 71484
all transformation are unique
all solve solve transformation are unique
all generated transformation have been solved


In [9]:
### Parameters of the transformations used and their total number

result = df.groupby(['min_sl', 'max_sl', 'm', 'H4', 'heuristic', 'option']).size().reset_index(name='count')

filtered_df_1 = result[(result['heuristic']=='1') & (result['option']=='bsl') & (result['H4']==1)]
total_sum_1 = filtered_df_1['count'].sum()

filtered_df_0 = result[(result['heuristic']=='1') & (result['option']=='bsl') & (result['H4']==0)]
total_sum_0 = filtered_df_0['count'].sum()

# Create a DataFrame to export as LaTeX
latex_table = pd.DataFrame({
    '\\texttt{min\\_sl}': list(filtered_df_1['min_sl']),
    '\\texttt{max\\_sl}': list(filtered_df_1['max_sl']),
    '\\texttt{m}': list(filtered_df_1['m']),
    'Total (\\(\\text{\\texttt{H}}_4 = 1\\))': list(filtered_df_1['count']),
    'Total (\\(\\text{\\texttt{H}}_4 = 0\\))': list(filtered_df_0['count'])
})

# Export to LaTeX
latex_output = latex_table.to_latex(index=False)

# Print or save the LaTeX table
print(latex_output)
print(total_sum_1,total_sum_0)

\begin{tabular}{rrrrr}
\toprule
\texttt{min\_sl} & \texttt{max\_sl} & \texttt{m} & Total (\(\text{\texttt{H}}_4 = 1\)) & Total (\(\text{\texttt{H}}_4 = 0\)) \\
\midrule
2 & 0 & 0 & 1670 & 14659 \\
2 & 2 & 0 & 411 & 3502 \\
2 & 2 & 1 & 101 & 1541 \\
2 & 2 & 2 & 170 & 4273 \\
2 & 2 & 3 & 158 & 4838 \\
3 & 3 & 0 & 414 & 3331 \\
3 & 3 & 1 & 101 & 1316 \\
3 & 3 & 2 & 167 & 3912 \\
3 & 3 & 3 & 161 & 4691 \\
4 & 4 & 0 & 417 & 3131 \\
4 & 4 & 1 & 101 & 1097 \\
4 & 4 & 2 & 170 & 3646 \\
4 & 4 & 3 & 161 & 4592 \\
5 & 5 & 0 & 409 & 3081 \\
5 & 5 & 1 & 101 & 968 \\
5 & 5 & 2 & 170 & 3456 \\
5 & 5 & 3 & 162 & 4406 \\
\bottomrule
\end{tabular}

5044 66440


In [10]:
### Statistics on the topology of transformed graphs

df = RESULTS
df = df[(df['option']=='bsl') & (df['heuristic']=='1')]
for h4 in [0, 1]:
    results = {}
    
    for tt in ['n_vertex', 'n_tolled', 'n_edge']:
        
        # Filter the DataFrame for the specific 'H4' value
        filtered_data = df[df['H4'] == h4][tt]
        
        # Calculate the required statistics
        results[tt] = {
            'Minimum': filtered_data.min(),
            'Médian': filtered_data.median(),
            'Variance':filtered_data.var(),
            'Moyenne': filtered_data.mean(),
            'Maximum': filtered_data.max()
        }

    # Create a new DataFrame from the results
    result_df = pd.DataFrame(results).T  # Transpose to have columns as 'min', 'median', 'mean', 'max'
    # result_df.index.name = 'Metrics'
    result_df = result_df.rename(index={
                                            'n_vertex': 'Sommets',
                                            'n_tolled': 'Arcs contrôlés',
                                            'n_edge': 'Arcs'
                                        })

    latex_table = result_df.to_latex(index=True, float_format="%.2f")
    print(latex_table)


\begin{tabular}{lrrrrr}
\toprule
 & Minimum & Médian & Variance & Moyenne & Maximum \\
\midrule
Sommets & 38.00 & 54.00 & 9.10 & 54.04 & 59.00 \\
Arcs contrôlés & 29.00 & 39.00 & 2.85 & 38.72 & 41.00 \\
Arcs & 193.00 & 203.00 & 2.85 & 202.72 & 205.00 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
 & Minimum & Médian & Variance & Moyenne & Maximum \\
\midrule
Sommets & 56.00 & 58.00 & 0.43 & 58.29 & 59.00 \\
Arcs contrôlés & 36.00 & 40.00 & 0.79 & 39.90 & 41.00 \\
Arcs & 200.00 & 204.00 & 0.79 & 203.90 & 205.00 \\
\bottomrule
\end{tabular}



In [11]:
### Number of instances solved by each combination of method and strategy for transformations constrained by H4
for h4 in range(2):
    dfs = []
    for key in RESULTS:
        if not key == 'combined':
                
            df = result_dict[key]['edge_value_df']
            df = df.drop(df.columns[0], axis=1)
    
            column_names = df.columns.tolist()
            data = [tuple(name.rsplit('-')[-6:]) for name in column_names]
            data = filter(lambda x: x[0]==str(h4), data)
         
            data = ['-'.join(c[-2::]) for c in data]

            unique_elements, counts = np.unique(data, return_counts=True)
    
            # Create a frequency dictionary
            freq_dict = dict(zip(unique_elements, counts))
            
            # Initialize an empty DataFrame
            methods = ['rnd', 'bsl',  'zip', 'min', 'avg', 'max']
            instances = [1, 2, 3, 4, 5, 6, 'x']
            table_data = {method: [freq_dict.get(f'{method}-{instance}', 0) for instance in instances] for method in methods}
            
            # Create DataFrame
            df = pd.DataFrame(table_data, index=instances)
            dfs.append(df.copy())

    
    result_df = sum(dfs)
    # Calculate total for each row and add as a new column
    result_df['Total'] = result_df.sum(axis=1)
    
    # Calculate total for each column and append as a new row
    result_df.loc['Total'] = result_df.sum()

    # Convert to LaTeX table
    latex_table = result_df.to_latex(index=True, float_format="%.2f")
    
    # Print the LaTeX code
    print(latex_table)



\begin{tabular}{lrrrrrrr}
\toprule
 & rnd & bsl & zip & min & avg & max & Total \\
\midrule
1 & 59470 & 66440 & 44979 & 66374 & 58359 & 58414 & 354036 \\
2 & 59447 & 66440 & 44996 & 66374 & 58360 & 58419 & 354036 \\
3 & 66205 & 66440 & 64555 & 66385 & 66404 & 66379 & 396368 \\
4 & 66167 & 66440 & 64575 & 66378 & 66411 & 66377 & 396348 \\
5 & 66205 & 66440 & 64555 & 66385 & 66404 & 66379 & 396368 \\
6 & 5340 & 0 & 10973 & 9247 & 9317 & 9430 & 44307 \\
x & 66213 & 0 & 64582 & 66402 & 66427 & 66398 & 330022 \\
Total & 389047 & 332200 & 359215 & 407545 & 391682 & 391796 & 2271485 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrr}
\toprule
 & rnd & bsl & zip & min & avg & max & Total \\
\midrule
1 & 4497 & 5044 & 3510 & 5038 & 4471 & 4538 & 27098 \\
2 & 4495 & 5044 & 3510 & 5038 & 4471 & 4538 & 27096 \\
3 & 4811 & 5044 & 4900 & 5038 & 5020 & 5037 & 29850 \\
4 & 4808 & 5044 & 4900 & 5038 & 5020 & 5037 & 29847 \\
5 & 4811 & 5044 & 4900 & 5038 & 5020 & 5037 & 29850 \\
6 & 2832 & 0 & 4034 

In [13]:
### Mean and variance of the relative increase in total resolution time, relative increase in resolution time and relative optimal gap for transformations unconstrained by H4


df = RESULTS['combined']['meta_data_df']
for h4 in range(2):
    filter_df = df[df['H4']==h4]
    for val in ['adjusted_time_increase', 'time_increase', 'optimal_gap']:
        print(h4, val)
        # Group by 'strategy' and 'method', then calculate mean and variance
        agg_df = filter_df.groupby(['heuristic', 'option'])[val].agg(['median', 'mean', 'std', 'var'])
        
        # Format the result as "mean ± variance"
        formatted_agg_df = agg_df.apply(lambda x: f"{x['mean']:.2f} ± {x['var']:.2f}", axis=1)
        
        # Reshape to have 'heuristic' as rows and 'option' as columns
        final_table = formatted_agg_df.unstack(level='option').fillna('')
        
        # Add Row Totals (mean ± variance across columns)
        row_totals = filter_df.groupby('heuristic')[val].agg(['mean', 'var'])
        row_totals_formatted = row_totals.apply(lambda x: f"{x['mean']:.2f} ± {x['var']:.2f}", axis=1)
        final_table['Total'] = row_totals_formatted
        
        # Add Column Totals (mean ± variance across rows)
        col_totals = filter_df.groupby('option')[val].agg(['mean', 'var'])
        col_totals_formatted = col_totals.apply(lambda x: f"{x['mean']:.2f} ± {x['var']:.2f}", axis=1)
        # Add an extra row for column totals at the bottom
        final_table.loc['Total'] = col_totals_formatted

        grand_total_mean = df[val].mean()
        grand_total_var = df[val].var()
        
        # Format the grand total for the bottom-right element
        grand_total_formatted = f"{grand_total_mean:.2f} ± {grand_total_var:.2f}"
        
        
        # Assign the grand total to the bottom-right cell
        final_table.at['Total', 'Total'] = grand_total_formatted


        final_table = final_table[['rnd', 'bsl',  'zip', 'min', 'avg', 'max', 'Total']]
        # Convert the final table to LaTeX format
        latex_table = final_table.to_latex(
            header=True,  # Include column headers
            index=True,   # Include row indices (heuristics)
            column_format='|c|' + 'c|' * (len(final_table.columns)-1),  # Adjust for added Total column
            escape=False  # Allow LaTeX special characters (like ±)
        )
        
        # Print or save the LaTeX table
        print(latex_table)

0 adjusted_time_increase
\begin{tabular}{|c|c|c|c|c|c|c|}
\toprule
option & rnd & bsl & zip & min & avg & max & Total \\
heuristic &  &  &  &  &  &  &  \\
\midrule
1 & -0.71 ± 0.16 & -1.00 ± 0.00 & -0.94 ± 0.07 & -0.82 ± 0.13 & -0.81 ± 0.14 & -0.82 ± 0.14 & -0.92 ± 0.13 \\
2 & -0.01 ± 0.25 & -0.98 ± 0.01 & -0.58 ± 0.14 & -0.40 ± 0.20 & -0.36 ± 0.21 & -0.38 ± 0.20 & -0.52 ± 0.25 \\
3 & -0.66 ± 0.18 & -0.67 ± 0.09 & -0.84 ± 0.11 & -0.74 ± 0.17 & -0.72 ± 0.18 & -0.72 ± 0.18 & -0.73 ± 0.16 \\
4 & -0.01 ± 0.26 & -0.97 ± 0.01 & -0.53 ± 0.15 & -0.23 ± 0.22 & -0.21 ± 0.23 & -0.23 ± 0.22 & -0.42 ± 0.27 \\
5 & -0.66 ± 0.18 & -0.67 ± 0.09 & -0.84 ± 0.11 & -0.74 ± 0.17 & -0.72 ± 0.18 & -0.72 ± 0.18 & -0.73 ± 0.16 \\
6 & 0.09 ± 0.16 &  & -0.17 ± 0.27 & 0.06 ± 0.25 & 0.06 ± 0.26 & 0.06 ± 0.25 & 0.04 ± 0.25 \\
x & -0.71 ± 0.16 &  & -0.93 ± 0.08 & -0.83 ± 0.13 & -0.81 ± 0.14 & -0.82 ± 0.14 & -0.84 ± 0.14 \\
Total & -0.43 ± 0.26 & -0.95 ± 0.07 & -0.79 ± 0.15 & -0.62 ± 0.21 & -0.60 ± 0.22 & -0.61 ± 0.21